In [ ]:
# Robotics control in a simulated environment
# Robotics Engineering
# 
# @file rt2_assignment_jupyter.ipynb
# @author Simone Contorno (@simone-contorno)
# @copyright Copyright (c) 2022

# Jupyter headers
import jupyros as jr
import ipywidgets as widgets
from IPython.display import display, clear_output

# ROS headers
import rospy
import actionlib
from geometry_msgs.msg import Twist
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal, MoveBaseActionGoal

pub_goal = rospy.Publisher('/move_base/goal', MoveBaseActionGoal, queue_size = 1000)
#client = actionlib.SimpleActionClient('move_base', MoveBaseAction)
#client.wait_for_server()
output = widgets.Output()
x = 0.0
y = 0.0
goal_flag = 0;
rospy.set_param('/goal_flag', goal_flag)
    
# Go back to the interface
def back(arg):
    clear_output() # Clear the cell output
    interface()
    
# Terminate the program
def terminate(arg):
    #client.wait_for_server()
    rospy.set_param('/goal_flag', 0)
    #client.cancel_goal()
    
    clear_output() # Clear the cell output
    print("Bye.")

# Send the new coordinates
def send_coordinates(arg):
    '''
    goal_pos = MoveBaseGoal()
    goal_pos.target_pose.header.frame_id = "map"
    goal_pos.target_pose.pose.orientation.w = 1
    goal_pos.target_pose.pose.position.x = x
    goal_pos.target_pose.pose.position.y = y
    '''
    goal_pos = MoveBaseActionGoal()    
    goal_pos.goal.target_pose.header.frame_id = "map"
    goal_pos.goal.target_pose.pose.orientation.w = 1;
    goal_pos.goal.target_pose.pose.position.x = x.value;
    goal_pos.goal.target_pose.pose.position.y = y.value;

    # Publish
    #client.wait_for_server()
    #client.send_goal(goal_pos)
    pub_goal.publish(goal_pos)
    rospy.set_param('/goal_flag', 1)
    
    with output:    
        output.clear_output()
        print("Goal sent.\n")

# Set new coordinated
def set_coordinates(arg):
    clear_output() # Clear the cell output
    print("Insert coordinates:")
    
    button = widgets.Button(description = 'Send') 
    back_button = widgets.Button(description = 'Back')
    
    global x
    global y 
    
    x = widgets.BoundedFloatText(
    value = 0.0,
    min = -100,
    step = 0.1,
    description = 'x :',
    disabled = False
    )
    y = widgets.BoundedFloatText(
    value = 0.0,
    min = -100,
    step = 0.1,
    description = 'y :',
    disabled = False
    )
    
    back_button.on_click(back)
    button.on_click(send_coordinates)
    
    display(x, y, button, back_button, output)

# Cancel the current goal
def cancel_goal(arg):
    goal_flag = rospy.get_param("/goal_flag")
    if goal_flag == 1:
        rospy.set_param('/goal_flag', 0)
        #client.cancel_goal()
        goal_flag = 0
        with output:    
            output.clear_output()
            print("Goal cancelled.\n")
    else: 
        with output:    
            output.clear_output()
            print("There is no goal set.\n")

# Show the UI
def interface():
    res = '5'
    
    # Manage printing
    flag = 0 
    
    # Choose the action to perform
    print("Choose an action:")
    exit = widgets.Button(description = 'Exit')
    goal = widgets.Button(description = 'Insert new coordinates to reach')
    canc = widgets.Button(description = 'Cancel the current goal')
    drive = widgets.Button(description = 'Manual driving')
    if flag == 0:
        with_assist = widgets.Button(description = 'Enable driving assistance')
    elif flag == 1:
        without_assist = widgets.Button(description = 'Disable driving assistance')
    
    exit.on_click(terminate)
    goal.on_click(set_coordinates)
    canc.on_click(cancel_goal)
    #drive.on_click()
    #with_assist.on_click()
    #without_assist.on_click()
    
    if flag == 0:
        display(exit, goal, canc, drive, with_assist)
    elif flag == 1:
        display(exit, goal, canc, drive, without_assist)
    
    '''
    # Manual driving
    elif res == '3':
        if goal_flag == 1:
            rospy.set_param('/goal_flag', 0)
            client.cancel_goal()
        (flag, counter1) = manualDriving(flag)

    # Enable/Disable driving assistance
    elif res == '4':
        if flag == 0:
            rospy.set_param('/drive_flag', 1)
            flag = 1
            print("\nDriving assistance enabled.\n")
        elif flag == 1:
            rospy.set_param('/drive_flag', 0)
            flag = 0
            print("\nDriving assistance disabled.\n")
    '''
    
def main():
    print("\nWelcome to the User Interface!\n"
        "Here you can choose between two different "
        "modalities to control your robot: automatic "
        "goal reaching or manual driving, with or without "
        "the driving assistance!")
    rospy.init_node("jupyter_interface")    
    interface()

if __name__ == '__main__':
    main()

In [ ]:
# Robotics control in a simulated environment
# Robotics Engineering
# 
# @file rt2_assignment_jupyter.ipynb
# @author Simone Contorno (@simone-contorno)
# @copyright Copyright (c) 2022

# Jupyter headers
import os
from jupyros import ros3d

view = ros3d.Viewer() # ROS 3D object
rc = ros3d.ROSConnection(url = "ws://localhost:9090") # Websocket (9090:rosbridge)
tf_client = ros3d.TFClient(ros = rc, fixed_frame = 'map') # Client connection 

# Visualize the map, the laser and the path to follow
map_view = ros3d.OccupancyGrid(topic = "/map", ros = rc, tf_client = tf_client) 
laser_view = ros3d.LaserScan(topic = "/scan", ros = rc, tf_client = tf_client) 
path = ros3d.Path(topic = "/move_base/NavfnROS/plan", ros = rc, tf_client = tf_client)

urdf = ros3d.URDFModel(ros = rc, tf_client = tf_client,
    path = os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000'))

g = ros3d.GridModel() # Grid object

# Show the model
view.objects = [g, laser_view, map_view, path, urdf]
view